Realizar el *Feature engineering* del dataset [MuestraDM](./materials/MuestraDM.csv) Para entregar el **12 de mayo del 2025 en su carpeta correspondiente del repositorio y presentarlo en clase el día 13 de mayo** . Subir a la carpeta `tareas/nombre` el notebook utilizado en tu *Feature engineering*.


In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
import random
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer


In [42]:
df = pd.read_csv('../output/DM_proc.csv')
df.head()

,newid,cx_curp,nota_medica,glucosa,colesterol,trigliceridos,hdl,ldl,fecha,presion_arterial,...,in_consulta,fecha_nacimiento,sexo,medicamentos,codigos_cie,diagnosticos,fecha_consulta,presion_sistolica,presion_diastolica,num_medicamentos
0,4593753CAAE51120998X96PC1,CAAE51120998X96PC1,NaN,135.0,0.0,0.0,0.0,0.0,NaN,120/60,...,4593753,1951-12-09,F,FLUOXETINA CAPSULA O TABLETA CADA CAPSULA O TA...,NaN,NaN,2014-12-29 09:02:48.167,120.0,60.0,4.0
1,2502589CAAE51120998X96PC1,CAAE51120998X96PC1,NaN,131.0,0.0,0.0,0.0,0.0,NaN,130/80,...,2502589,1951-12-09,F,"COMPLEJO B| METFORMINA, TABLETAS, METFORMINA, ...",NaN,NaN,2010-12-21 10:58:00.640,130.0,80.0,2.0
2,6253338CAAE51120998X96PC1,CAAE51120998X96PC1,NaN,148.0,0.0,0.0,0.0,0.0,NaN,120/80,...,6253338,1951-12-09,F,INSULINA HUMANA ACCION INTERMEDIA NPH SUSPENSI...,NaN,NaN,2017-10-16 10:09:10.830,120.0,80.0,8.0
3,6866604CAAE51120998X96PC1,CAAE51120998X96PC1,NaN,0.0,0.0,0.0,0.0,0.0,NaN,130/70,...,6866604,1951-12-09,F,FLUOXETINA CAPSULA O TABLETA CADA CAPSULA O TA...,NaN,NaN,2018-11-02 10:22:01.733,130.0,70.0,12.0
4,2870758CAAE51120998X96PC1,CAAE51120998X96PC1,ELIZABET DE 59.9 AÑOS EDAD---ACUDE A EVALCUION...,126.0,0.0,0.0,0.0,0.0,2011-09-01,NaN,...,2870758,1951-12-09,F,COMPLEJO B| PRAVASTATINA. TABLETAS. CADA TABLE...,E119,Diabetes tipo II,2011-10-08 09:47:14.953,0.0,0.0,6.0


In [43]:
col_escalar = ['glucosa', 'colesterol','trigliceridos','hdl','ldl','hba1c','plaquetas','creatinina',
			   'urea','peso','altura','imc','presion_sistolica','presion_diastolica','num_medicamentos']
# Escalado de variables numéricas
def scale_numeric(df, cols):
	scaler = StandardScaler()
	df[cols] = scaler.fit_transform(df[cols])
	
	return df

df = scale_numeric(df, col_escalar)

In [44]:
#data profiling
# Data profiling de variables númericas
def get_repeated_values(df, col, top):
	"""Return top 't' of repeated values."""
	top_5 = df.groupby([col])[col]\
					.count()\
					.sort_values(ascending = False)\
					.head(5)
	indexes_top_5 = top_5.index
	
	if ((top == 1) and (len(indexes_top_5) > 0)):
		return indexes_top_5[0]
	elif ((top == 2) and (len(indexes_top_5) > 1)):
		return indexes_top_5[1]
	elif ((top == 3) and (len(indexes_top_5) > 2)):
		return indexes_top_5[2]
	else: 
		return 'undefined'

def numeric_profiling(df_pkl, col):
	"""
	Profiling for numeric columns. 
	
	:param: column to analyze
	:return: dictionary
	"""
	profiling = {}

	profiling.update({'max': df_pkl[col].max(),
					 'min': df_pkl[col].min(),
					 'mean': df_pkl[col].mean(),
					 'stdv': df_pkl[col].std(),
					 '25%': df_pkl[col].quantile(.25),
					 'median': df_pkl[col].median(),
					 'mode': df_pkl[col].mode(),
					 '75%': df_pkl[col].quantile(.75),
					 'kurtosis': df_pkl[col].kurt(),
					 'skewness': df_pkl[col].skew(),
					 'uniques': df_pkl[col].nunique(),
					 'valores_unicos': df_pkl[col].nunique(),
					 'renglones_totales': df_pkl[col].size,
					 'faltantes_totales': df_pkl[col].isna().sum(),
					 'prop_missings': df_pkl[col].isna().sum()/df_pkl.shape[0]*100,
					 'top1_repeated': get_repeated_values(df_pkl, col, 1),
					 'top2_repeated': get_repeated_values(df_pkl, col, 2),
					 'top3_repeated': get_repeated_values(df_pkl, col, 3)})    
	
	return profiling



numeric_profiling_output = {elem: numeric_profiling(df, elem)\
							 for elem in col_escalar}

df_numeric_profiling = pd.DataFrame(numeric_profiling_output).reset_index()
df_numeric_profiling.rename(columns={'index': 'metric'}, inplace=True)
df_numeric_profiling


,metric,glucosa,colesterol,trigliceridos,hdl,ldl,hba1c,plaquetas,creatinina,urea,peso,altura,imc,presion_sistolica,presion_diastolica,num_medicamentos
0,max,7.263365,7.09661,18.281571,15.725607,16.120193,96.492522,26.915875,86.296522,27.224017,2.753725,1.319679,8.358027,2.276387,2.654766,25.458326
1,min,-0.460584,-0.247683,-0.220291,-0.108173,-0.090349,-0.012806,-0.042976,-0.077094,-0.090269,-0.98499,-1.024998,-0.228383,-0.962276,-0.96175,-1.259771
2,mean,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0
3,stdv,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000054,1.000059
4,25%,-0.460584,-0.247683,-0.220291,-0.108173,-0.090349,-0.012806,-0.042976,-0.077094,-0.090269,-0.98499,-1.024998,-0.228383,-0.962276,-0.96175,-0.492009
5,median,-0.460584,-0.247683,-0.220291,-0.108173,-0.090349,-0.012806,-0.042976,-0.077094,-0.090269,0.21674,0.720767,-0.228383,-0.962276,-0.96175,-0.184905
6,mode,"0 -0.460584 Name: glucosa, dtype: float64","0 -0.247683 Name: colesterol, dtype: float64","0 -0.220291 Name: trigliceridos, dtype: float64","0 -0.108173 Name: hdl, dtype: float64","0 -0.090349 Name: ldl, dtype: float64","0 -0.012806 Name: hba1c, dtype: float64","0 -0.042976 Name: plaquetas, dtype: float64","0 -0.077094 Name: creatinina, dtype: float64","0 -0.090269 Name: urea, dtype: float64","0 -0.98499 Name: peso, dtype: float64","0 -1.024998 Name: altura, dtype: float64","0 -0.228383 Name: imc, dtype: float64","0 -0.962276 Name: presion_sistolica, dtype: ...","0 -0.96175 Name: presion_diastolica, dtype: ...","0 -0.492009 Name: num_medicamentos, dtype: f..."
7,75%,-0.460584,-0.247683,-0.220291,-0.108173,-0.090349,-0.012806,-0.042976,-0.077094,-0.090269,0.991188,0.956509,-0.228383,0.980922,1.10483,0.275752
8,kurtosis,6.442671,14.665067,62.926388,100.556435,138.597543,9308.61204,579.738039,5993.489042,537.872705,-1.703693,-1.9655,18.237732,-1.892652,-1.894363,153.083989
9,skewness,2.416713,3.97585,6.681269,9.811619,11.639655,96.464658,23.953144,71.133396,21.032418,0.169566,-0.031486,4.364256,0.119225,0.121238,7.344245


In [45]:

df = df.dropna(subset=['sexo'])

#  Limpiamos y homogenizamos
df = df.dropna(subset=['sexo'])
df['sexo'] = df['sexo'].str.strip().str.upper()

# Configurar encoder para manejar posibles nuevas categorías
encoder = OneHotEncoder(
	sparse_output=False,
	drop='first', # Elimina la primera categoría para evitar la multicolinealidad   
	handle_unknown='ignore'
)

# Codificar
encoded_array = encoder.fit_transform(df[['sexo']])
column_names = encoder.get_feature_names_out(['sexo'])

# Crear el DataFrame codificado
encoded_df = pd.DataFrame(encoded_array, columns=column_names, index=df.index)

# Integrar al DataFrame original
df = pd.concat([df.drop(columns=['sexo']), encoded_df], axis=1)
df.to_csv('../output/DM_proc_enconded.csv', index=False)

